# TASK 4: Color
Support colour sequences taking advantage of the chromatic components from different color spaces.

In [8]:
from VehicleDetection import VehicleDetection
import os
import cv2
import numpy as np
import matplotlib.pyplot as pltÇ
import random
import copy
import statistics
from eval_utils import *
from video_utils import *
from load_utils import *
from background_remover import *

data_path = '../datasets/AICity_data/train/S03/c010/'

In [9]:
gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')

Try RGB, HS from HSV, UV from YUV, AB from LAB

In [10]:
# @ 30s UOC Server
color_spaces = {"RGB": cv2.COLOR_BGR2RGB,"HSV": cv2.COLOR_BGR2HSV, "LAB": cv2.COLOR_BGR2LAB, 'YUV': cv2.COLOR_BGR2YUV}
means, stds = get_background_stats_color(data_path + 'vdo.avi', 1, 411) # 411 is the 20% of the frames

100%|██████████| 410/410 [00:11<00:00, 36.93it/s]


In [11]:
def remove_background_adaptative_color(means, stds, videoPath, ROIpath, alpha=4, sigma=2, p=0.1, color="RGB"):
    roi = cv2.imread(ROIpath, cv2.IMREAD_GRAYSCALE)
    vidcap = cv2.VideoCapture(videoPath)
    num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    detections = {}
    for frame in tqdm(range(num_frames)):
        _, image = vidcap.read()
        if frame >= num_frames // 4:
            # Read Color Image
            img_color = cv2.cvtColor(image, color_spaces[color])
            # Split Channels
            img_color_C1 = img_color[:, :, 0]
            img_color_C2 = img_color[:, :, 1]
            img_color_C3 = img_color[:, :, 2]
            # Mask per Channel
            img_mask_C1 = np.zeros(img_color_C1.shape)
            img_mask_C2 = np.zeros(img_color_C2.shape)
            img_mask_C3 = np.zeros(img_color_C3.shape)
            # Compute mask with formula
            if color == "RGB":
                img_mask_C1[abs(img_color_C1 - means[0]) >= alpha * (stds[0] + sigma)] = 255
                img_mask_C2[abs(img_color_C2 - means[1]) >= alpha * (stds[1] + sigma)] = 255
                img_mask_C3[abs(img_color_C3 - means[2]) >= alpha * (stds[2] + sigma)] = 255
                img_mask = img_mask_C1 + img_mask_C2 +img_mask_C3
                # AND: just where all channels agree
                img_mask[img_mask == 255*3] = 255
                img_mask[img_mask != 255] = 0

            print(img_mask.shape)
            print(roi.shape)
            cleaned = cleanMask(img_mask, roi)
            cv2.imwrite(f'./masks_adaptative_RGB/mask_{frame}.png', cleaned)

            #update mean
            idxs = cleaned == 0
            means[0][idxs] = p * img_color_C1[idxs] + (1 - p) * means[0][idxs]
            means[1][idxs] = p * img_color_C2[idxs] + (1 - p) * means[1][idxs]
            means[2][idxs] = p * img_color_C3[idxs] + (1 - p) * means[2][idxs]
            #update std
            stds[0][idxs] = np.sqrt(p * (img_color_C1[idxs] - means[0][idxs])**2 + (1 - p) * stds[0][idxs]**2)
            stds[1][idxs] = np.sqrt(p * (img_color_C2[idxs] - means[1][idxs])**2 + (1 - p) * stds[1][idxs]**2)
            stds[2][idxs] = np.sqrt(p * (img_color_C3[idxs] - means[2][idxs])**2 + (1 - p) * stds[2][idxs]**2)

            detections[str(frame)] = getBoxesFromMask(cleaned)

    return detections

In [12]:
singleGaussianDetections = remove_background_adaptative_color(means,stds, data_path + 'vdo.avi',data_path + 'roi.jpg',4,2,0.07, "RGB")

 25%|██▍       | 534/2141 [00:02<00:09, 173.66it/s]

(1080, 1920)
(1080, 1920)


 25%|██▍       | 535/2141 [00:03<00:10, 156.41it/s]


error: OpenCV(4.5.5) /io/opencv/modules/core/src/arithm.cpp:212: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'binary_op'


In [ ]:
recs, precs, aps = ap_wo_conf(gt_detect,singleGaussianDetections)
plot_prec_recall_curve(np.mean(precs, axis=0), recs[0], f'Precision-Recall curve for Adatative Gaussian  - AP {np.mean(aps)}')